# ANOVA - Analysis of Variance
Today, we will be learning ANOVA, a generalized form of comparing mean across multiple groups. 
Agenda today:
1. Compare t-tests and ANOVA
2. Differentiate between variance between groups and variance within groups
3. Implement ANOVA in Python
    - using scipy
    - from scratch
    - using statsmodels
        - Discuss the relationship between ANOVA and Linear Regression.
4. Demo of a post hoc test (Tukey Test)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

## T tests or ANOVA?
**ANOVA** or *Analysis Of Variance*  provides a statistical test of whether two or more population means are equal, and therefore generalizes the t-test beyond two means.

Suppose we want to compare whether multiple groups differ in some type of measures. For example, we have collected mood data grouped by four types of weather - sunny, raining, overcast, or cloudy, and we want to find out whether there is a difference in mood across different weather. What tests would you use?

A natural reaction would be to conduct multiple t-tests. However, that comes with many drawbacks. First, you would need $\frac{n(n-1)}{2}$ t tests, which come out to 6 tests. Having more tests meaning having higher chance of making type I error. In this case, our original probability of making type I error grew from 5% to 5% x 6 = 30%! By conducting 6 tests and comparing their mean to each other, we are running a huge risk of making false positives. This is known as the multiple comparison problem. 

Let's demonstrate this below:

In [ ]:
# Set random seed for reproducibility
np.random.seed(2021)
# Create a numpy array with 100 observations
p_values = np.zeros(100)
# Run a for loop with 100 iterations
for iteration in range(100):
    # Create a normal distribution
    population = stats.norm(0,1)
    # Draw two samples from the same distribution
    first_sample = population.rvs(50)
    second_sample = population.rvs(50)
    # Run a ttest on the data to evaluate
    # whether there is significant difference
    # between samples
    test = stats.ttest_ind(first_sample, second_sample)
    # Collect the pvalue from the test
    pvalue = test.pvalue
    # Round the pvalue for comparison
    pvalue = round(pvalue, 2)
    # Add the pvalue to the numpy array
    p_values[iteration] = pvalue

# Count the number of times a pvalue is less than
# or equal to .05
false_positives = (p_values <= .05).sum()

# Print the number of times we found
# a significant difference between samples
# drawn from the same population.
print('Out of 100 tests, we had {} false positives.'.format(false_positives))

How then, can we combat this? -- ANOVA!

Instead of looking at each individual difference, ANOVA examines the ratio of variance between groups, and variance within groups, to find out whether the ratio is big enough to be statistically significant. 

#### T Test statistics 
##### One sample
$t = \frac{x\bar - \mu}{\frac{s}{\sqrt n}}$

##### Two sample
$$ t = \frac{\bar{x_1} - \bar{x_2}}{\sqrt{s^2 \left( \frac{1}{n_1} + \frac{1}{n_2} \right)}}$$

where $s^2$ is the pooled sample variance,

$$ s^2 = \frac{\sum_{i=1}^{n_1} \left(x_i - \bar{x_1}\right)^2 + \sum_{j=1}^{n_2} \left(x_j - \bar{x_2}\right)^2 }{n_1 + n_2 - 2} $$

We can also say that t test is a special case of ANOVA in that we are comparing the means of only two groups.


#### ANOVA - the F test
$F = \frac{MS_{bet}}{MS_{within}}$

Just like t and z tests, we calculate a test statistic, then compare it to a critical value associated with a probability distribution.  In this case, that is the f-distribution.

![fdistribution](images/f_distribution.png)

Degrees of freedom of an F-test originate from:
  - the degrees of freedom from the numerator of the f-stat (DF between)
  - the degrees of freedom from the denominator of the f-stat (DF within) 
(more below)

**When working with two samples, ANOVA is really just a "special" ttest.**

Below, let's create two samples drawn from a normal distribution

*Sample 1*
- Mean: 0
- Standard Deviation: 3
- Size: 100

*Sample 2*
- Mean: 1
- Standard Deviation: 3
- Size: 100

In [ ]:
# CODE HERE

Next, we will run an ANOVA test using the `f_oneway` function from the `scipy.stats` module.

In [ ]:
# CODE HERE

Now, let's run a ttest using the `ttest_ind` function from the `scipy.stats` module.

In [ ]:
# CODE HERE

**What do we notice about these two test results?**

Is the t-statistic related to the f-statistic?

In [ ]:
# CODE HERE

# Discussion:

## Which test would you run for each these scenarios:

1. The average salary per month of an English Premier League player is $240,000€$. You would like to test whether players who don't have a dominant foot make more than the rest of the league.  There are only 25 players who are considered ambidextrous. 

2. You would like to test whether there is a difference in arrest rates across neighborhoods with different racial majorities.  You have point statistics of mean arrest rates associated with neighborhoods of majority white, black, hispanic, and asian populations.

3. You are interested in testing whether the superstition that black cats are bad luck affects adoption rate.  You would like to test whether black-fur shelter cats get adopted at a different rate than cats of other fur colors.

4. You are interested in whether car-accident rates in cities where marijuana is legal differs from the general rate of car accidents.  Assume you know the standard deviation of car accident rates across all U.S. cities.



## 2. Differentiate between variance between groups and variance within groups


<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/12/image046.png" width="500">

#### Between Group Variability

Measures how much the means of each group vary from the mean of the overall population



<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/12/image0171.png" width="500">
    

#### Within Group Variability

Refers to variations caused by differences within individual groups.  

Consider the given distributions of three samples below. As the spread (variability) of each sample is increased, their distributions overlap and they become part of a big population.
<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/12/image031.png" width="500">

Now consider another distribution of the same three samples but with less variability. Although the means of samples are similar to the samples in the above image, they seem to belong to different populations.

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/12/image033.png" width="500">


### Which of these following trios would have high f-stats, and which would have low.

In [ ]:
# Create three sets of data without much difference in means
np.random.seed(42)

a = np.random.normal(20,20,20)
b = np.random.normal(22,20,20)
c = np.random.normal(19,20,20)

one = np.random.normal(20,2,20)
two = np.random.normal(22,2,20)
three = np.random.normal(19,2,20)

four = np.random.normal(20,10,20)
five = np.random.normal(20,10,20)
six = np.random.normal(23,10,20)

In [ ]:
groups = [[a,b,c], [one,two,three], [four,five,six]]
fig, axes = plt.subplots(1,3, figsize=(15,3))

for idx in range(len(groups)):
    ax = idx%3
    for group in groups[idx]:
        sns.kdeplot(group, ax = axes[ax], shade=True)
        axes[ax].set_title('Group {}'.format(idx + 1))

Let's see how ANOVA identifies significant difference between these three examples.

In [ ]:
print(stats.f_oneway(a,b,c))
print(stats.f_oneway(one,two,three))
print(stats.f_oneway(four,five,six))

## Calculating ANOVA

In this section, we will learn how to calculate ANOVA ourselves.

In [ ]:
df = pd.read_csv('data/bikeshare_day.csv')
df.head()
# cnt is the outcome we are trying to predict

In [ ]:
# we need to conduct a little bit feature engineering to encode 
df['season_cat'] = df.season.apply(lambda x: 'spring' if x == 1 else 
                                           (
                                            'summer' if x == 2 else (
                                                'fall' if x == 3 else 'winter')
                                           )
                                      )
df.head()

In [ ]:
#Create a boxplot
# 1 is spring, 2 is summer, 3 is fall, and 4 is winter
df.boxplot('cnt', by='season_cat', figsize=(6,6))

In [ ]:
# We could perform two sample t-tests for each sample.

# subset the dataframe  by season and isolate the dependent variable
spring = df[df.season_cat == 'spring'].cnt
fall = df[df.season_cat == 'fall'].cnt
summer = df[df.season_cat == 'summer'].cnt
winter = df[df.season_cat == 'winter'].cnt

# We could run independent t-tests for each combination
# But that increases the chance of making a type I (False Positive) error
# Also, as your groups increase, the number of tests may become infeasable
print(stats.ttest_ind(spring, fall))
print(stats.ttest_ind(spring, summer))
print(stats.ttest_ind(spring, winter ))
print(stats.ttest_ind(fall, summer))
print(stats.ttest_ind(fall, winter))
print(stats.ttest_ind(summer, winter))

#### Breakdown of the calculation

Total Sum of Squares is the square of every value minus the mean means, or in other words, the variance of the entire dataset without dividing through by degrees of freedom. 
- $SS_t$ = $\sum (X_{ij} - \bar X)^2$

The total sum of squares can be broken down into the sum of squares between and the sum of squares within.
- $SS_t =  SS_b+SS_w  $

The sum of squares between accounts for variance in the dataset that comes from the difference between the mean of each sample, without dividing through by the degrees of freedom.   
Or, in other words, the weighted deviation of each mean from the mean of means:
- $SS_b$ = $\sum(n_i(\bar X - \bar X_i)^2) $

The sum of squares within accounts for variance that comes from within each sample.  That is, the sum of the variance of each group weighted by its degrees of freedom. This is really just the sum of the square of each data point's deviation from its sample mean:
- $SS_w$ = $\sum (n_i - 1) s_i ^ 2$  

Degrees of Freedom for ANOVA:
-  $DF_{between}$ = k - 1
- $DF_{within}$ = N - k
- $DF_{total}$ = N - 1

Notations:
- $\bar{X} = $ Mean of Means = Mean of entire dataset
- $X_{i}$ = An individual Sample
- $X_{ij}$ = An observation in an individual sample
- $\bar{X_{i}}$ = The mean of an individual sample
- k is the number of groups
- N is the total number of observations
- n is the number of observations in each group
- $MS$ = Mean Sum of Squares
- $DF$ = Degrees of Freedom
- $F$ = F Statistic
- $_b$ = between
- $_w$ = within

- $MS_b$ = $\frac{SS_b}{DF_b}$
- $MS_w$ = $\frac{SS_w}{DF_w}$


- $F$ = $\frac{MS_b}{MS_w}$

Like t-tests, we can also perform hypothesis testing with ANOVA. 

- $H_0$ : $\mu{_1}$ = $\mu_2$ = $\mu_3$ = $\mu_4$
- $H_a$ : $H_0$ is not true

Under the null hypothesis (and with certain assumptions), both quantities estimate the variance of the random error, and thus the ratio should be small. If the ratio is large, then we have evidence against the null, and hence, we would reject the null hypothesis.

**Quick intro to flexible arguments**

In Python, the print function can take any number of arguments:

In [ ]:
print(1,2,3,4,5)
print(1)
print('First object', 2, 'third object')
print(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)

We can write functions that work this way too. 

To do so, we place a `*` in front of a placeholder variable when we define the function. 

In [ ]:
def example(*stuff):
    return stuff

In [ ]:
example(1,2,3,4,5)

In [ ]:
def example(*stuff):
    count = 0
    for item in stuff:
        count += item
    return count

In [ ]:
example(1,2,3,4,5)

Because it makes out lives easier, we will apply the same principal to the our `anova` function.

**Let's write some *code***

The goal of our function is to output a dataframe that replicates this table, which is produced by statsmodels ANOVA.

|            | df    | sum_sq       | mean_sq      | F          | PR(>F)       |
|------------|-------|--------------|--------------|------------|--------------|
| season_cat | 3.0   | 9.505959e+08 | 3.168653e+08 | 128.769622 | 6.720391e-67 |
| Residual   | 727.0 | 1.788940e+09 | 2.460715e+06 | NaN        | NaN          |

In [ ]:
def anova(*samples, group_label='samples'):
    # Merge all samples into a single array
    
    # Calculate the mean of all samples
    
    # Calculate the sum of square difference
    # between all observations and the overall_mean
    
    # Calculate the sum of squared differences 
    # within each sample
    
    # Calculate the sum of squared differences
    # between the overall mean and each sample mean
    # weighted by the size of each sample
#===================================================================
          # If the weighting in the line above is confusing,
          # you could optionally just loop over each individual
          # data point in a sample and calculate the difference
          # between the overall mean and the sample mean ^squared
          # forever data point. The calculation is the same.
            
#         for observation in sample:
#             ss_between += (overall_mean - sample.mean())**2
#===================================================================    
    # Calculate the numbers of groups
    
    # Calculation the total numbers of observations
    
    # Calculate the degrees of freedom within 
    
    # Calculate the mean squared difference within
    
    # Calculate the degrees of freedom between
    
    # Calculate the mean suared difference between
    
    # Calculate the f statistic
    
    # Calculate the p_value
    
    

    column_names = ['df', 'sum_sq', 'mean_sq', 'F', 'PR(>F)']
    data = [[df_between, ss_between, ms_between, round(f, 6), p_value],
            [df_within,  ss_within,  ms_within, np.nan, np.nan]]

    df = pd.DataFrame(data, columns=column_names, index=[group_label, 'Residual'])

    return df

In [ ]:
anova(summer, fall, winter, spring, group_label = 'Seasons')

## Perform an ANOVA with scipy

In [ ]:
anova = stats.f_oneway(df['cnt'][df['season_cat'] == 'summer'],
                df['cnt'][df['season_cat'] == 'fall'], 
                df['cnt'][df['season_cat'] == 'winter'],
                df['cnt'][df['season_cat'] == 'spring'])

anova.statistic

## 4. Calculate ANOVA using statsmodel

In [ ]:
df.boxplot('cnt', by = 'season_cat')

In [ ]:
linear_regression_model = ols('cnt~season_cat',data=df).fit()

In [ ]:
# Run Anova with the linear regression model
anova_table = sm.stats.anova_lm(linear_regression_model)
print(anova_table)

Why are we using linear regression for ANOVA?

We haven't gotten to Linear Regression yet, but let's take a look at the table reporting the coefficients for linear regression. 

In [ ]:
# Collect the summary tables from the linear regression model
regression_table = linear_regression_model.summary()
# Collect the second table which contains the coefficients
regression_table = regression_table.tables[1]
# Convert the data into a dataframe
regression_table = pd.DataFrame(regression_table.data[1:],columns = regression_table.data[0])
regression_table.head()

If we look at first column, we see the following values:

```Intercept, season_cat[T.spring], season_cat[T.summer], season_cat[T.winter]```

These values represent each of the seasons. In this case, `fall` has been set as the intercept. 

Let's calculate the mean for each of the seasons in our dataset and append them to the results table. 

In [ ]:
means = df.groupby('season_cat')['cnt'].mean()

In [ ]:
regression_table['means'] = means.values

Now let's subtract the fall mean from all other mean values, and save the output as another column. 

In [ ]:
regression_table['mean-fall-mean'] = regression_table.means - regression_table.means[0]

In [ ]:
regression_table

We can see that the difference between fall and the other season mean is the same as their linear regression coefficient! 

## Next steps
Just because we have rejected the null hypothesis, it doesn't mean we have conclusively showed which group is significantly different from which - remember, the alternative hypothesis is "the null is not true". 

We need to conduct post hoc tests for multiple comparison to find out which groups are different, the most prominent post hoc tests are:
- LSD (Least significant difference)
    - $t\sqrt \frac{MSE}{n^2}$
- Tukey's HSD 
    - $q\sqrt \frac{MSE}{n}$
    
https://www.statisticshowto.com/studentized-range-distribution/#qtable
    
After calculating a value for LSD or HSD, we compare each pair wise mean difference with the LSD or HSD difference. If the pairwise mean difference exceeds the LSD/HSD, then they are significantly different.

In [ ]:
results = pairwise_tukeyhsd(df.cnt, df.season_cat)

In [ ]:
results.summary()